In [1]:
#importando as bibliotecas 
import pandas as pd
import json
import nltk
import time
import re
from bs4 import BeautifulSoup
nltk.download('punkt')
from textblob import TextBlob
import sys
import os.path

[nltk_data] Downloading package punkt to /home/pcorrea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
inicio = 17506
terminou = 20000

In [3]:
! git clone https://github.com/paulordie/sentimental_ds

fatal: destination path 'sentimental_ds' already exists and is not an empty directory.


In [4]:
%cd sentimental_ds/
%ls

/home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/sentimental_ds
dados-curso.csv        main.txt           piloto-2.ipynb
dataset-v2.dat         pc-dataset-v2.csv  prototype_sentimental_analyzer.ipynb
first_code_test.ipynb  pc-dataset-v2.dat  README.md


In [5]:
import json
import pandas as pd
import time
dados = open('dataset-v2.dat', encoding="utf8") #pandas
print("Tipo do dado: ", type(dados))

Tipo do dado:  <class '_io.TextIOWrapper'>


In [6]:
! pip install googletrans

In [7]:
listOfDataBase = list()
listOfData = []
 
def findKeys(dataBase):
    for register in dataBase:
        try:
            cols = register.split('\n')
            listOfDataBase.append(cols[0])
        except UnicodeDecodeError:
            print("Error on uncode decode findkeys")
    print("Find Keys: ",listOfDataBase)

In [8]:
def passToJson():
    i=0
    for i in range(100000): ## tem que colocar a array de todo arquivo
        listOfIssue = listOfDataBase[i]
        i+=1
#         time.sleep(1)
        joinList = json.loads(listOfIssue)
        data = pd.json_normalize(joinList)
        listOfData.append(data)
    return listOfData

In [9]:
def showContent(value, collum):
    i=0
    _data = []
    for i in range(value): ##range que vai trazer todas as colunas
        try:
            res = listOfData[i][collum]
            print(type(res))
            print(listOfData[i][collum]) #averageRating, ratingValue etc
            i = i + 1
            print("line: ", i)
        except KeyError:
            print("Key Error")
            continue

valueFindKeys = findKeys(dados) #valor que vai para def passToJson()
dictionaryOfDate = passToJson()
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
df = pd.concat(objs=dictionaryOfDate,axis=0)

In [11]:
X = df.iloc[:, 4].values
type(X)
len(X)

100000

In [27]:
X

array(['Comida maravilhosa, lugar do verdadeiro chiclete de camarão. \nO garçom Éric Lima nos atendeu muito bem, pretendemos voltar assim que possível.',
       'Comida boa, drinks legais e um ambiente bem aconchegante. \nProvamos muitas coisas e tudo nos agradou. Principalmente um pato servido em um dos pratos da casa, sensacional!',
       'Excelente pra ir com a família! Ótima Comida, ambiente agradável, bom atendimento e estacionamento. Vale a pena conhecer',
       ...,
       'O Rio Scenarium é uma das baladas mais famosas do Rio de Janeiro. É bem grande e lotada de estrangeiros. A bebida não é tão cara como em outras cidades, por exemplo, São Paulo. O lado negativo é com certeza a fila. Se você não chegar cedo, vai ficar pelo menos 30 minutos esperando para entrar.',
       'Fui por indicação e com boa expectativa!\n116,00 o kilo de salada... Bem caro, né?\nO alface tava murcho, nada de opções de tempero, buffet pequeno...\n12,00 um copo peq de suco. Aff.\nNao valeu, nem pelo am

In [13]:
def clean_tripadvisor(tweet):
    try:
        tweet = BeautifulSoup(tweet, 'lxml').get_text() # o lxml indica o parse pode ser usado para dados de web por ex
    #     tweet = re.sub(r"\b\.\s\\n", '\n', tweet)
    #     tweet = re.sub(r"\b\.\s[A-Za-z0-9]", '\n', tweet)
        tweet = re.sub(r"\b\.\s\\n", '. \n', tweet)
        tweet = re.sub(r"\b\.", '. \n', tweet)    
        return tweet
    except TyperError:
        print("Errou no tipo no regex")

In [14]:
arquivoToSave = "/home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-{}-{}.txt".format(inicio, terminou)

def saveDocument(var,resultado,nota,polaridade,fim):
    try:
        with open(arquivoToSave, 'a') as toSave:
            toSave.write('{} - {} subj {} pol {} \n'.format(var,resultado,nota, polaridade))
            if fim == "fim":
                toSave.close()
            print('It was created result: ',arquivoToSave)
                
    except Exception as e:
        print("Error did file ", e)
            

In [15]:
arquivoToSaveError = "/home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/erros_poradiradade_vm2-{}-{}.txt".format(inicio, terminou)

def saveDocumentError(var, item,obj):
    try:
        with open(arquivoToSaveError, 'a') as toSave:
            toSave.write('{} - qtd erro {} : {} \n'.format(var,item,obj))
            
            toSave.close()
            print('It was saver error: ',arquivoToSaveError)
                
    except Exception as e:
        print("Error did file ", e)

In [16]:
def transTextBlob(sentences):
    ##textblob
    comentario_blob = TextBlob(sentences)
    print(type(comentario_blob))
    comentario_blob_us = comentario_blob.translate(to='us')
    print("textblob traduziu: ",comentario_blob_us)
    return comentario_blob, comentario_blob_us
    ##textblob

In [17]:
def transGoogle(sentences):
    ##googletrans
    import pandas as pd
    from googletrans import Translator   
    translator = Translator()
    try:
        translations = translator.translate([sentences], dest='en')
        for translation in translations:
            print(type(translation.text))
            print("google traduziu: ", translation.text)
    except AttributeError:
         print("Error googletrans")
    ##googletrans

In [30]:
var = 0
errou = 0
for x in df['reviewBody']:
    ini = time.time()
    var = var + 1
    if var > inicio and var <= terminou:
        print("**************\n")
        print( "var {} x {}".format(var,x))
        print("**************\n")
        #out = clean_tripadvisor("Recife: a beautiful city with a gorgeous culture. Beyond the beauties, we also have a really good food. So, do you really want to eat an awesome steak? If you love steak like I do, you need to know a perfect place. What’s the name of the place? Spettus! You’ll see more than 22 kinds of steaks, all of them prepared by excellent professionals. Steaks from Argentina and Uruguay, and you can choose any kind of steak you want. You’ll find japanese food, vegetables and delicious deserts too! What are you going to find there? A huge place (they fit 700 people), perfect for a good lunch, a dinner with the family and, of course, a happy hour with your friends. Do you like good music? Piano bar. Do you have kids? Playground. Everything you’ll find at Spettus! Do you drink? Be prepared to drink a good wine from France, Italy, Argentina and more! Oh! Do you want to surf on the internet while eating? Do you need an Internet Wi-fi? Spettus is the answer! I went there last year and it was perfect. I love steak and they have a really good variety. And, obvious, it’s delicious! Remember: Spettus!")
        out = clean_tripadvisor(x)
        #################################
        try:
            which = TextBlob(out)
            print("Tipo de which language: ",type(which))
            lang = which.detect_language()
            if lang == 'en':
                print("It is english")
                #################################
                #comentario_blob = TextBlob(out)
    #             print("comentario_blob ",type(which))

                comentario_blob_pt = which.translate(to='pt')
                comentario_blob_us = comentario_blob_pt.translate(to='en')
                print("tipo em english ", type(which))
                print("Original em Inglês: ",which)
                print("Traduzido para Português: ",comentario_blob_pt)
                time.sleep(5)

                for sentence_br, sentence in zip(comentario_blob_pt.sentences, comentario_blob_us.sentences):
                    print("sentence_br ",sentence_br)
                    print(" EN sentence.sentiment.polarity: ",sentence.sentiment.polarity)
                    print("EN sentence.sentiment.subjectivity: ",sentence.sentiment.subjectivity)
                    if sentence_br is None:
                        errou = errou + 1
                        print("errou em {} - {} {}".format(var,errou,x))
                        saveDocumentError(var,errou,x)

                    elif var > inicio and var <= terminou:
                        print('Polaridade\n',sentence_br, ':', sentence.sentiment.polarity)
                        print('subjetividade\n',sentence_br, ':', sentence.sentiment.subjectivity)
                        saveDocument(var,sentence_br,sentence.sentiment.subjectivity,sentence.sentiment.polarity, 'fim')

                    else:
                        print("EN algum erro ocorreu em {}".format(var))
                ################################################

            elif lang == 'pt':
                print("It is portuguese")
                #################################
                comentario_blob = TextBlob(out)
                print("comentario_blob ",type(comentario_blob))

                comentario_blob_us = comentario_blob.translate(to='us')
                print("tipo em english ", type(comentario_blob_us))
                print("Inglês: ",comentario_blob_us)
                time.sleep(5)

                for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
                    print("sentence_br ",sentence_br)
                    print("sentence.sentiment.polarity: ",sentence.sentiment.polarity)
                    print("sentence.sentiment.subjectivity: ",sentence.sentiment.subjectivity)
                    if sentence_br is None:
                        errou = errou + 1
                        print("errou em {} - {} {}".format(var,errou,x))
                        saveDocumentError(var,errou,x)

                    elif var > inicio and var <= terminou:
                        print('Polaridade\n',sentence_br, ':', sentence.sentiment.polarity)
                        print('subjetividade\n',sentence_br, ':', sentence.sentiment.subjectivity)
                        saveDocument(var,sentence_br,sentence.sentiment.subjectivity,sentence.sentiment.polarity, 'fim')

                    else:
                        print("PT algum erro ocorreu em {}".format(var))
                ################################################
            else:
                print("Not english")
        except TypeError:
            print("Errou não é tipo string")
            continue
        
    if var == terminou:
        print("Finalizou em ",var)
        break
    fim = time.time()
    print("tempo final foi ", ini-fim)
   

tempo final foi  -1.9073486328125e-06
tempo final foi  -1.1920928955078125e-06
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final f

tempo final foi  -7.152557373046875e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo fin

tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo

tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final 

tempo final foi  -4.76837158203125e-07
tempo final foi  -1.430511474609375e-06
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -1.6689300537109375e-06
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -2.384185791015625e-07
tempo 

...

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  How I love this place.
 Varied snacks, vegetarian options, french fries, among others.
 Not to mention that the environment is very cool.
 
My only caveat is that ketchup and mustard are always running out.
..
sentence_br  Como eu adoro esse lugar.
sentence.sentiment.polarity:  0.5
sentence.sentiment.subjectivity:  0.6
Polaridade
 Como eu adoro esse lugar. : 0.5
subjetividade
 Como eu adoro esse lugar. : 0.6
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-11210-20000.txt
sentence_br  Lanches variados, opções vegetarianas, batata-frita, entre outros.
sentence.sentiment.polarity:  0.0
sentence.sentiment.subjectivity:  0.0
Polaridade
 Lanches variados, opções vegetarianas, batata-frita, entre outros. : 0.0
subjetividade
 Lanches variados, opções vegetarianas, batata-frita, entre ou

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  I used Barranco for the last time about 18 years ago, despite the fact that several times I passed through the city I never had the opportunity to go to that nice house again, and I am frank in saying that I missed it, especially with the pork lover. , the nostalgia fell on such a pork loin with cheese, a friend told me: if you don't know how to choose ok I will take you to the bank and there I went to miss you, better off I would have missed you.
 - Barranco is the same friendly place, the service in general is from reasonable to bad, which saddened me, once the order is over there is no more sadness, the disappointment was total.
 The loin with cheese was nothing like the old one, it was badly made, it looked like cooked meat, dry hard without taste and the cheese guy even raised suspicion about the amount of breadcrumbs mixed.
 I am frank to admit that no, th

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  Great option to try a shrimp carvery or a more sophisticated seafood dish.
 The service is cordial, the environment very beautiful and you will have a view of the sea while eating.
 Price a little salty, but in the end, the cost benefit pays off.
 I recommend!
sentence_br  Ótima opção para provar um rodízio de camarão ou um prato de frutos do mar mais sofisticado.
sentence.sentiment.polarity:  0.6
sentence.sentiment.subjectivity:  0.75
Polaridade
 Ótima opção para provar um rodízio de camarão ou um prato de frutos do mar mais sofisticado. : 0.6
subjetividade
 Ótima opção para provar um rodízio de camarão ou um prato de frutos do mar mais sofisticado. : 0.75
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-11210-20000.txt
sentence_br  O atendimento é cordial, o ambiente muito boni

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  Very good, very reasonable price and very beautiful place.
 I can't say that I fell in love, but it's a restaurant that has everything to do with Christmas.
 I recommend.
sentence_br  Muito bom, preço bastante razoável e lugar muito bonito.
sentence.sentiment.polarity:  0.7233333333333333
sentence.sentiment.subjectivity:  0.8533333333333334
Polaridade
 Muito bom, preço bastante razoável e lugar muito bonito. : 0.7233333333333333
subjetividade
 Muito bom, preço bastante razoável e lugar muito bonito. : 0.8533333333333334
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-11210-20000.txt
sentence_br  Não posso dizer que me apaixonei, mas é um restaurante que tem tudo haver com Natal.
sentence.sentiment.polarity:  0.5
sentence.sentiment.subjectivity:  0.6
Polaridade
 Não posso dizer q

HTTPError: HTTP Error 429: Too Many Requests